In [1]:
from pandas import read_csv
import tensorflow as tf
import numpy as np
from google.colab.drive import mount
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model


In [2]:
mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd  '/content/drive/MyDrive/BigData'

/content/drive/MyDrive/BigData


In [4]:
df = read_csv('./features.csv')
df.head()

,ID,Label,Sequence,hydrophobicity_polar_first,hydrophobicity_polar_20%,hydrophobicity_polar_40%,hydrophobicity_polar_60%,hydrophobicity_polar_80%,hydrophobicity_polar_100%,hydrophobicity_neutral_first,...,solvent_accessibility_intermediate_40%,solvent_accessibility_intermediate_60%,solvent_accessibility_intermediate_80%,solvent_accessibility_intermediate_100%,solvent_accessibility_exposed_first,solvent_accessibility_exposed_20%,solvent_accessibility_exposed_40%,solvent_accessibility_exposed_60%,solvent_accessibility_exposed_80%,solvent_accessibility_exposed_100%
0,WP_395757889.1,AMP,MPYDSVYLEKRPPGALRTVWRKFYGDTTAMIGLYGCAGLLLLCVFG...,0.013514,0.189189,0.304054,0.631757,0.804054,1.000000,0.006757,...,0.371622,0.540541,0.736486,0.932432,0.013514,0.189189,0.304054,0.631757,0.804054,1.000000
1,WP_395755974.1,AMP,MKILIVEDDTLIQQGLAQAMARENYAYDCADSAVGASALLQSSHYS...,0.009091,0.254545,0.440909,0.636364,0.831818,0.990909,0.045455,...,0.377273,0.645455,0.845455,1.000000,0.009091,0.254545,0.440909,0.636364,0.831818,0.990909
2,WP_395732316.1,AMP,MDFSRKPTQDPEAWLDFAPEFSQSMARQVRGWIQRWEPDLAESIKW...,0.009852,0.182266,0.418719,0.605911,0.862069,0.995074,0.019704,...,0.399015,0.630542,0.783251,0.950739,0.009852,0.182266,0.418719,0.605911,0.862069,0.995074
3,WP_395723680.1,AMP,MDCSRTPTRDPETWLDLAPEFSQAMARQVRAWIQRWEPDLTESIKW...,0.009852,0.192118,0.477833,0.655172,0.866995,0.995074,0.019704,...,0.325123,0.497537,0.689655,0.921182,0.009852,0.192118,0.477833,0.655172,0.866995,0.995074
4,WP_395695240.1,AMP,MADRDTAGVTFETTVSSFGNNTGIEVPEEVIERLGRGKRPPVLVTV...,0.019231,0.205128,0.455128,0.717949,0.891026,1.000000,0.012821,...,0.346154,0.532051,0.762821,0.884615,0.019231,0.205128,0.455128,0.717949,0.891026,1.000000


In [5]:
df['Label'] = df['Label'].map({'nAMP': 0, 'AMP': 1})

In [7]:
y = df['Label'].values
y

array([1, 1, 1, ..., 0, 0, 0])

In [8]:
# Các tham số

#max_length = 200
amino_acids = "ACDEFGHIKLMNPQRSTVWXY"
num_amino_acids = len(amino_acids)
dic_to_index = {val: i + 1 for i, val in enumerate(amino_acids)}
index_to_dic = {i + 1: val for i, val in enumerate(amino_acids)}

In [9]:
sequences = df['Sequence'].values
def encode_sequence(sequence):
    return [dic_to_index[nuc] for nuc in sequence]

encoded_sequences = [encode_sequence(seq) for seq in sequences]

max_length = max([len(seq) for seq in encoded_sequences])

for i in range(len(encoded_sequences)):
    if len(encoded_sequences[i]) < max_length:
        encoded_sequences[i].extend([0] * (max_length - len(encoded_sequences[i])))
X = np.array(encoded_sequences)


In [9]:
X.shape

(6000, 9624)

In [10]:
# Step 2: Define the model in TensorFlow
# We will use an Embedding layer followed by an LSTM layer to learn sequence-level embeddings

# Parameters
embedding_dim = 32  # Dimensionality of the embedding vector

lstm_units = 16    # Number of LSTM units (so nay sua khi nao vay?)

# Define the model
model = tf.keras.Sequential([
    # Embedding Layer
    tf.keras.layers.Embedding(input_dim = num_amino_acids + 1, output_dim = embedding_dim, input_length = max_length),

    # LSTM Layer
    #tf.keras.layers.LSTM(lstm_units, return_sequences=False),  # return_sequences=False to output a single vector
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units = lstm_units)),

    # Dense Layers for classification (optional, depends on your task)
    tf.keras.layers.Dense(units = 16, activation='relu'),
    tf.keras.layers.Dense(units = 1, activation='sigmoid')  # Example output layer (for classification)
])

# Step 3: Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Step 4: Train the model (dummy labels for illustration purposes)
# For this example, let's assume the sequences are labeled for a classification task (e.g., 0, 1, 2, 3)
#labels = np.array([0, 1, 2])

# Train the model


# Step 5: Get the embeddings for a sequence
# sequence = "ATGCAAA"  # Example sequence
# encoded_sequence = encode_sequence(sequence)  # Encode the sequence
# encoded_sequence = np.expand_dims(encoded_sequence, axis=0)  # Add batch dimension

# # Get the embedding output
# embedding_output = model.predict(encoded_sequence)

# print(f"Embedding for sequence {sequence}: {embedding_output}")
#model.load_weights('./model.keras')


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [11]:
model.fit(X, y, epochs=1, batch_size=2)


3000/3000 ━━━━━━━━━━━━━━━━━━━━ 1343s 445ms/step - accuracy: 0.6640 - loss: 0.5986


In [13]:
model.save('./model.keras')

In [16]:
model2= load_model('./model.keras')

In [12]:
model.load_weights('./model.keras')

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 24 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


ValueError: A total of 5 objects could not be loaded. Example error message for object <Embedding name=embedding, built=True>:

Layer 'embedding' expected 1 variables, but received 0 variables during loading. Expected: ['embeddings']

List of objects that could not be loaded:
[<Embedding name=embedding, built=True>, <LSTMCell name=lstm_cell, built=True>, <LSTMCell name=lstm_cell, built=True>, <Dense name=dense, built=True>, <Dense name=dense_1, built=True>]

In [17]:
model2.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (2, 9624, 32)               │             704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (2, 32)                     │           6,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (2, 16)                     │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (2, 1)                      │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 22,565 (88.15 KB)

 Trainable params: 7,521 (29.38 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 15,044 (58.77 KB)

In [18]:
model2.fit(X, y, epochs=2, batch_size=2)

Epoch 1/2
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 1323s 441ms/step - accuracy: 0.7571 - loss: 0.4819
Epoch 2/2
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 1374s 451ms/step - accuracy: 0.7777 - loss: 0.4412


In [19]:
model2.save('./model2.keras')

In [11]:
model3= load_model('./model2.keras')

In [12]:
model3.fit(X, y, epochs=2, batch_size=2)

Epoch 1/2
   8/3000 ━━━━━━━━━━━━━━━━━━━━ 14:05:05 17s/step - accuracy: 0.8301 - loss: 0.3430

KeyboardInterrupt: 

In [ ]:
model3.save('./model3.keras')

In [ ]:
model4= load_model('./model3.keras')

In [ ]:
model4.fit(X, y, epochs=2, batch_size=2)

In [ ]:
word_embeddings = model.layers[0].get_weights()[0]
word_embeddings

array([[-4.17071015e-01, -1.24115519e-01, -1.49331465e-01,
         7.37697557e-02, -1.15560785e-01,  2.48767555e-01,
        -8.63146782e-02, -4.44302619e-01, -2.00531378e-01,
         2.63448834e-01,  3.04616749e-01, -4.74540710e-01,
        -3.82408738e-01,  3.29024523e-01, -6.09661676e-02,
        -1.14171252e-01, -2.49507781e-02, -3.12163323e-01,
         1.18696123e-01,  9.31427069e-03, -9.97548625e-02,
        -1.71857357e-01, -8.06789100e-03, -3.07558417e-01,
         1.81628074e-02, -1.01732716e-01,  7.67685026e-02,
         1.34791672e-01, -4.34260339e-01, -1.97423264e-01,
         3.39236230e-01,  3.91510725e-02],
       [ 7.48454332e-02,  9.08282027e-02,  9.71152857e-02,
         1.32094637e-01,  1.11891776e-01, -1.14703812e-01,
        -1.72838420e-02,  7.84312040e-02, -6.97102249e-02,
        -2.40747705e-02,  4.49001901e-02, -4.14129570e-02,
         8.86866450e-02, -2.80827936e-02,  1.51382834e-01,
        -7.25354180e-02, -7.84625560e-02,  7.79066533e-02,
         3.53

In [ ]:
#elasstic search =>
#search Data Warehouse